In [326]:
from IPython.display import HTML, display
import pandas as pd
from collections import Counter
import statistics
from collections import OrderedDict

import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.collocations import *
from nltk.corpus import names 


#file_path = "/Users/Sofia/Desktop/LING_506/project/data/cancer_research_coping_with_loss.csv"
#file_path = "/Users/Sofia/Desktop/LING_506/project/data/cancer_research_dying_with_cancer.csv"
#file_path = "/Users/Sofia/Desktop/LING_506/project/data/cancer_research_living_with_cancer.csv"
file_path = "/Users/Sofia/Desktop/LING_506/project/data/cancer_research_caring.csv"

coping_loss_data = pd.read_csv(file_path, delimiter="\t")
column_names = ['Thread_name', 'Thread_address', 'Username', "Post_id",
                                  "Date", "Post_text", "Reply_to_id"]

#result = coping_loss_data.tail(10)
#print(result)

In [327]:
# DESCRIPTIVE STATISTICS

# TOTAL NUMBER OF THREADS:  
print(len(coping_loss_data.Thread_name.unique()), "total number of threads")

# TOTAL NUMBER OF POSTS: 
print(len(coping_loss_data), "total number of posts")

# TOTAL NUMBER OF USERS
print(len(coping_loss_data.Username.unique()), "total number of users", "\n")

#user_arr = list(coping_loss_data["Username"])
#unique_users = set(user_arr)
#for user in unique_users:
#    print(user)


# MAX, MIN, AVERAGE NUM OF POSTS PER THREAD ----------------------------------

thread_arr = list(coping_loss_data["Thread_name"])
thread_post_count = Counter(thread_arr) # count how many times each thread has a post
thread_post_count = dict(sorted(thread_post_count.items(), key=lambda item: item[1]))

#for thread, post_count in thread_post_count.items():
#    print(thread, post_count)

num_posts_thread = list(thread_post_count.values())

print(max(num_posts_thread), "max num of posts per thread")
print(min(num_posts_thread), "min num of posts per thread")
print(statistics.mean(num_posts_thread), "average num of posts per thread")
print(statistics.mode(num_posts_thread), "mode of num of posts per thread", "\n")



# MAX, MIN, AVERAGE NUM OF POSTS PER USER ----------------------------------

# count how many times user appears in rows
user_arr = list(coping_loss_data["Username"])
user_post_count = Counter(user_arr) # count how many times each user has a post
user_post_count = dict(sorted(user_post_count.items(), key=lambda item: item[1]))

#for user, post_count in user_post_count.items():
#    print(user, post_count)

num_user_posts_thread = list(user_post_count.values())

print(max(num_user_posts_thread), "max num of posts per user")
print(min(num_user_posts_thread), "min num of posts per user")
print(statistics.mean(num_user_posts_thread), "average num of posts per user")
print(statistics.mode(num_user_posts_thread), "mode of num of posts per user", "\n")


# MAX, MIN, AVERAGE NUM OF THREADS PER USER ----------------------------------

user_arr = list(coping_loss_data["Username"]) # list of users
thread_arr = list(coping_loss_data["Thread_name"]) # list of threads
user_set = list(set(user_arr)) # list of unique users

user_thread_dict = dict() #key: user, value: list of unique threads where they posted

for i in range(len(user_arr)):

    if user_arr[i] in user_thread_dict.keys(): # check for user key
        if thread_arr[i] not in user_thread_dict[user_arr[i]]: # check if thread is in user's value list
            user_thread_dict[user_arr[i]].append(thread_arr[i])
    else:
        user_thread_dict[user_arr[i]] = [thread_arr[i]]
        
#for user, threads in user_thread_dict.items():
#    print(user, threads)


user_num_thread_dict = dict() #key: user, value: num of unique threads where they posted

for user in user_arr:
    user_num_thread_dict[user] = len(user_thread_dict[user])

user_num_thread_dict = dict(sorted(user_num_thread_dict.items(), key=lambda item: item[1]))
    
#for user, threads in user_num_thread_dict.items():
#    print(user, threads)

num_user_threads_thread = list(user_num_thread_dict.values())

print(max(num_user_threads_thread), "max num of threads per user")
print(min(num_user_threads_thread), "min num of threads per user")
print(statistics.mean(num_user_threads_thread), "average num of threads per user")
print(statistics.mode(num_user_threads_thread), "mode of num of threads per user", "\n")


4245 total number of threads
21122 total number of posts
5696 total number of users 

1365 max num of posts per thread
1 min num of posts per thread
4.975736160188457 average num of posts per thread
2 mode of num of posts per thread 

1026 max num of posts per user
1 min num of posts per user
3.7082162921348316 average num of posts per user
1 mode of num of posts per user 

594 max num of threads per user
1 min num of threads per user
2.259129213483146 average num of threads per user
1 mode of num of threads per user 



In [261]:
# EARLIEST, LATEST FORUM POST 

date_arr = list(coping_loss_data["Date"]) # list of dates

date_arr = sorted(date_arr)

years = []

for date in date_arr:
    date_split = date.split(" ")
    year = date_split[2]
    years.append(year)
    

test_date = date_arr[0]
test_date_split = test_date.split(" ")
#print(test_date_split)

years = sorted(years)

#for year in years:
#    print(year)

print(min(years), "earliest post - year")
print(max(years), "latest post - year")
print(statistics.mode(years), "year with most posts", "\n")


# DISTRIBUTION OF POSTS BY YEAR?

2008 earliest post - year
2021 latest post - year
2019 year with most posts 



### Text post stats - num of sentences / post

In [262]:
text_posts = list(coping_loss_data["Post_text"])
test_text = text_posts[0]

# POST LENGTH - NUM OF SENTENCES - MAX, MIN, AVERAGE
tokenized_text = sent_tokenize(test_text)

# "....." not recognized as sentence boundary

sent_num_arr = []
sent_length_arr = [] # count length of each sentence in words

tokenizer = RegexpTokenizer(r'\w+') # remove punctuation

count = 0
for text in text_posts:
    
    #print(count)
    
    if isinstance(text, str):
        text_sent = sent_tokenize(text) # list of sentences
        
        for sentence in text_sent:
            sent_to_word = tokenizer.tokenize(sentence) # tokenize sentence into words
            sent_length_arr.append(len(sent_to_word))
            
        
        sent_num_arr.append(len(text_sent))

        count += 1
        
print(max(sent_num_arr), "max num of sentences per post")
print(min(sent_num_arr), "min num of sentences per post")
print(statistics.mean(sent_num_arr), "average num of sentences per post")
print(statistics.mode(sent_num_arr), "mode of num of sentences per post", "\n")

# SENTENCE LENGTH NUM OF WORDS - max, min, average

print(max(sent_length_arr), "max num of words per sentence")
print(min(sent_length_arr), "min num of words per sentence")
print(statistics.mean(sent_length_arr), "average num of words per sentence")
print(statistics.mode(sent_length_arr), "mode of num of words per sentence", "\n")


#print(len(tokenized_text))
#for sent in tokenized_text:
#    print(sent)


77 max num of sentences per post
0 min num of sentences per post
5.315371642466373 average num of sentences per post
1 mode of num of sentences per post 

1288 max num of words per sentence
0 min num of words per sentence
22.217393323118355 average num of words per sentence
10 mode of num of words per sentence 



### Text post stats - num of words / post

In [263]:
# POST LENGTH - NUM OF WORDS - MAX, MIN, AVERAGE
tokenized_word = word_tokenize(test_text)

corpus_all_words = []

tokenizer = RegexpTokenizer(r'\w+') # remove punctuation
tokenized_word = tokenizer.tokenize(test_text)

#print(len(tokenized_word))
#for word in tokenized_word:
#    print(word)

word_num_arr = []

#count = 0
for text in text_posts:
    
    #print(count)
    
    if isinstance(text, str):
        text_word_num = tokenizer.tokenize(text)
        word_num_arr.append(len(text_word_num))
        
        corpus_all_words.extend(text_word_num)

        #count += 1

print(max(word_num_arr), "max num of words per post")
print(min(word_num_arr), "min num of words per post")
print(statistics.mean(word_num_arr), "average num of words per post")
#print(statistics.mode(word_num_arr), "mode num of words per post", "\n")

mode_counter = Counter(word_num_arr)
print(mode_counter.most_common(1), "mode num of words per post", "\n")

1858 max num of words per post
0 min num of words per post
118.09370243922504 average num of words per post
[(26, 359)] mode num of words per post 



### Text post stats - corpus word count

In [264]:
print(len(corpus_all_words), "total words in forum")

5741952 total words in forum


### Text post stats - top unigrams

In [265]:
# TOP UNIGRAMS - FREQUENCY
filtered_corpus_all_words = []

ps = PorterStemmer()
lem = WordNetLemmatizer()

stop_words = set(stopwords.words("english"))

for word in corpus_all_words:
    if word not in stop_words: # remove stop words
        
        lower_word = word.lower() # lowercase
        stem_word = lem.lemmatize(lower_word) # word lemma
        filtered_corpus_all_words.append(stem_word)

#print(corpus_all_words[0:50])
#print(filtered_corpus_all_words[0:50])

unigrams_dist = FreqDist(filtered_corpus_all_words)

# Compare 1st post in thread vs. replies?

print(len(corpus_all_words), "total words w/out stop words")
print(len(filtered_corpus_all_words), "total words, lowercase, w/out stop words")


5741952 total words w/out stop words
3003362 total words, lowercase, w/out stop words


In [227]:
#for unigram in unigrams_dist.most_common(100):
    #print(unigram[0], "\t", unigram[1])
    #print(unigram[1])

### Text post stats - top bigrams

In [255]:
corpus_bigrams = nltk.bigrams(filtered_corpus_all_words)
all_bigram_freq = nltk.FreqDist(corpus_bigrams)


In [226]:
#for bigram in all_bigram_freq.most_common(50):
    #print(bigram[0][0] + " " + bigram[0][1], "\t", bigram[1])
    #print(bigram[0][0] + " " + bigram[0][1])
    #print(bigram[1])

### Users’ relation to the cancer patient - thread names

In [275]:
thread_arr = list(set(list(coping_loss_data["Thread_name"]))) # unique thread names in corpus

relations = ["mother", "mom", "mum", "father", "dad", "daddy", "mommy",
            "sister", "brother", "grandparent", "grandmother", "grandfather", "grandad",
             "grandpa", "grandma", "son", "daughter", "granddaughter", "grandson",
             "niece", "nephew", "uncle", "aunt", "husband",
             "wife", "boyfriend", "girlfriend", "fiance", "fiancé", "partner"]

#sorted_relations = sorted(relations, key=len)

relation_counts = [] # count how many threads contain relation mention
friend_counts = 0 # count how many threads mention 'friend'

for line in thread_arr:
    
    if "friend" in line:
        friend_counts += 1
    
    for relation in relations:
        if relation in line:
            relation_counts.append(relation)
            #print(line)
            #print(relation, "\n")
            
relation_counts = Counter(relation_counts)                

print(friend_counts)

#for title in relations:
#    for relation, count in sorted(relation_counts.items()):
#        if relation == title:
            #print(relation, count)
            #print(count)    


90


### Usernames: inferring gender

In [328]:
user_arr = list(set(list(coping_loss_data["Username"])))

for user in user_arr:
    print(user)

willowgarden
lob
pats
donnamay
joeyc
cc-0
nmac234
shjoemma4
mags23
dawnangela
zig
becs
parky49
dafra
north27
justmadesoup
mhb
lisadav74
marco9987564
eddie16
iain1958
charlotte6875
anahita
maria55
shazag
bree
flyon92
other-half
avaarchie
jamlie
delma
narasimhans
mancpike
imabloke
samantha1994
suzy83
davefromhull
helen119
pandoram
debsspain
yare
glen66
mynan
struggler
hiddenshells
grana
samsunguk
angel1130
matildagooley
papasgirl
sindy75
feelingnumb
megsmom
hummer
jasmin21
boggit64
charlotte07
natypop121
midlandsvn
givemestrength
supportingdad
brian-lea
custardcream
38mstage4cancerparent
jocy
dna
quilterler
hellsbells66
mousie
eamccann
jackal
alan1977
sunflowerchild
claire1984
claireb112
kay13
suz448
dlux71
windmill
pure2000
martin17
khaddouj
lr92
sue12fm
markyn
katieb123
mellynoz
gemzy2019k
nika
helpful42
linsey46
amy15
midge
rachael53
iza
hothead49
klm1984
piglet38
eileemay
magg50
scooby78
jeffinee
alexm1985
rupertthebear
elmo007
adelebw
fraser
california
nickturner90
dollparts
tracyjb

mrsmc1976
cruk-nurse-vanda
tanyar
janwlf
nanuk
trish007
kathryn39
annaliisa
cases18
suzie801
tca
troffer123
auntyb
nico39
51yearsoflove
kays
blue76
penguin85
darcyc
alovedonewithcancer
sad2000
mandys-0
vadams
nessemo3
hina
pstacey1956
joanne88
ash919
isaac27
vikkianna
margoogram
kbob123
joanie1944
it-doesnt-feelfair
abiside
swook
kirstynic
jennieb
cgflear
cgregory
lizize
kathy-23
filwain100
twinkle14
fjh95
mygirl-0
moderator-jane
stompy
jillmoff
katherinelove
lexilou86
robins-0
aarti
wl
berry
friday
exilenumber53
collibosher
zoe98
brian1050
nikki23
paulaanne
kp6458
maof4
jellyfishy23
evey11
dc64
hisgirl
odea
jo08
traceyab
helsbels
sasha21
laura2709
sarahohlondon
rbarts
aelv94
pambuck01
hope1973
caztrace
jbicker
medi
mcclellanfirst
drummer-45
pmcgoohan
carrieev
suzannah
lucyda
bryonyo
dodge2018
joz
suwei
worriedmom
leigh1980
mhmartinez
sos
katewcx
katymac
isabelle75
sheilamary
rem
esther11
adry
laurenlab1
bobheather
butters1980
mrmoon
fe
4634biker
cassilda
shoe2190
tabby2020
neilp
tigge

butterflyangel
pinknotts
bridget002
welshcrocus
ginaandme
lostsister
napoleonicgamer
cassier
michellel
alfielurcher
caroline83
del
neganega10
emma83
jezzabell37
rayro
rebels
overarainbow
hh52
sammielou
rocker
noodles89
emily112
wongster
mazmaz
reet75
helenca
elsal29
kitkat1984
prufrock
jackiem777
vplum
nic219
fightingfordad
firsttimer
jassie
lizzie12447
xxleanne123x
lisamk
bubs74
jl03
alicevalladolid
autumnpumpkin
mumof1
lockie
loobyloobylou
rp
scottie456
mandym74
jlee1984
starcatone
alvin94
bcross
paulineaj
linz2009
dpm249
torty
claired
frazzle
boomoon
200bob
janeabeth
hu
anomonus223
jacintarella
rw37
wolds
dante000
chrissyb
alexisaxx
bellasg
bhab
jackie91
jo-10
1984pf
haywood02
happydaize
averaz
huytip
samw
hondacbr
star101
sodes
caz57
mart57
1lauren9
jansi
fraggle
swiftbird
saffronx
kacperw
louise513
peipei94
vickyd
ibo181
4todayonly
kevh
claudia1971
ojdoingmybest
euterpe
mike1977
crazycat69
dixxie75
luckyg
jamie34
dbic1029
charlie14
minithemynx
alinne
loulabellab
tara123
elizc
catl

In [304]:
female_name_file = open(r"/Users/Sofia/Desktop/LING_506/project/data/name_list/female_names.txt","r")
male_name_file = open(r"/Users/Sofia/Desktop/LING_506/project/data/name_list/male_names.txt","r")

female_name_db = female_name_file.read().split("\n")
male_name_db = male_name_file.read().split("\n")


female_name_db_lower = [] # female name database - by Kantrowitz, Ross
male_name_db_lower = [] # male name database - by Kantrowitz, Ross

gender_neutral_names = [] # appear in both female and male lists

for line in female_name_db:
    if not line.startswith("#") and line:
        female_name_db_lower.append(line.lower())
        #print(line.lower())
        
for line in male_name_db:
    if not line.startswith("#") and line:
        male_name_db_lower.append(line.lower())
        #print(line.lower())
        
        if line.lower() in female_name_db_lower: # check if name is gender neutral
            gender_neutral_names.append(line)
        

print(len(female_name_db_lower), "all female names")
print(len(female_name_db_lower) - len(gender_neutral_names), "only-female names", "\n")

print(len(male_name_db_lower), "all male names")
print(len(male_name_db_lower) - len(gender_neutral_names), "only-male names")


#for gend_neut in gender_neutral_names:
#    print(gend_neut)

# name lists into a list of ordered pairs: 


5001 all female names
4636 only-female names 

2943 all male names
2578 only-male names


In [306]:
female_users = []
non_female_users = []

count = 0
for user in user_arr:
    female_count = 0
    
    for fem_name in female_name_db_lower:
        if fem_name in user:
            female_count += 1
            
            #print(user, fem_name)
            
    if female_count > 0:
        female_users.append(user)
        
    else:
        non_female_users.append(user)

    #print(count, female_count, "\n")
    count += 1

        

In [295]:
print(len(female_users))
print(len(non_female_users))

2692
1612


### Moderator users

In [329]:
moderator_count = 0
moderators = []

for user in user_arr:
    if 'moderator' in user:
        moderator_count += 1
        moderators.append(user)
        print(user)
        
print("\n", moderator_count, "moderators in forum")

moderator-lucie
moderator-renata
moderatorrenata
moderator-jane
moderator-sarah
moderator-steph
moderator-ben
moderator-anastasia
moderator-helen
moderator-thea
moderator-kirsty
moderator-jenn

 12 moderators in forum


In [330]:
user_post_count = Counter(coping_loss_data.Username)
moderator_post_counts = []

for mod in moderators:
    
    moderator_post_counts.append(user_post_count[mod])
    print(mod, user_post_count[mod])


print("\n", statistics.mean(moderator_post_counts), "average num of posts by moderator")



moderator-lucie 390
moderator-renata 154
moderatorrenata 7
moderator-jane 94
moderator-sarah 26
moderator-steph 621
moderator-ben 136
moderator-anastasia 64
moderator-helen 45
moderator-thea 1
moderator-kirsty 4
moderator-jenn 343

 157.08333333333334 average num of posts by moderator
